# QBOB |👷🏾‍♂️> examples

## Hello world

In [1]:
from qbob import qbob, qpam
from qbob.intrinsics import Message

In [2]:
my_qbob = qbob.OperationBuilder("HelloWorld", entrypoint=True)
my_qbob += Message("Hello World!")

In [3]:
print(my_qbob.build())

@EntryPoint()
operation HelloWorld () : Unit {
    Message("Hello World!");
}


In [4]:
hello_world = my_qbob.compile()

Preparing Q# environment...
........

In [5]:
hello_world()

Hello World!


()

## QPAM |👷🏻‍♀️>

In [6]:
my_qpam = qpam.ProgramArchitect("MyProject")
my_qpam.add_operations(my_qbob)
my_qpam.create_project(".")

'MyProject.csproj'

In [7]:
!cat Program.qs

namespace MyProject {
    open Microsoft.Quantum.Canon;
    open Microsoft.Quantum.Intrinsic;

    @EntryPoint()
    operation HelloWorld () : Unit {
        Message("Hello World!");
    }
}

In [8]:
!dotnet run

========Hello World!
=

## Teleport

In [8]:
from qbob.intrinsics import H, CNOT, X, Z, Measure
from qbob.types import Qubit, Pauli, One

teleport = qbob.OperationBuilder("Teleport")
msg = teleport.input("msg", Qubit)
target = teleport.input("target", Qubit)

with teleport.allocate_qubits("auxiliary", 1) as aux:
    teleport += H(aux)
    teleport += CNOT(aux, target)
    teleport += CNOT(msg, aux)
    teleport += H(msg)

    with teleport.if_statement(Measure([Pauli.PauliZ], [msg]) == One):
        teleport += Z(target)

    with teleport.if_statement(Measure([Pauli.PauliZ], [aux]) == One):
        teleport += X(target)
        
print(teleport.build())

operation Teleport (msg : Qubit, target : Qubit) : Unit {
    using (auxiliary = Qubit()) {
        H(auxiliary);
        CNOT(auxiliary, target);
        CNOT(msg, auxiliary);
        H(msg);

        if (Measure([PauliZ], [msg]) == One) {
            Z(target);
        }

        if (Measure([PauliZ], [auxiliary]) == One) {
            X(target);
        }
    }
}


In [9]:
teleport.compile()

<Q# callable Teleport>

In [10]:
my_qbob = qbob.OperationBuilder("TeleportMessage")
message = my_qbob.input("message", bool)

with my_qbob.allocate_qubits("register", 2) as register:
    msg = register[0]
    target = register[1]

    with my_qbob.if_statement(message == "true"):
        my_qbob += X(msg)
    my_qbob += H(msg)

    my_qbob += teleport(msg, target)
    my_qbob.returns(Measure([Pauli.PauliX], [target]))
        
print(my_qbob.build())

operation TeleportMessage (message : Bool) : Result {
    using (register = Qubit[2]) {

        if (message == true) {
            X(register[0]);
        }
        H(register[0]);
        Teleport(register[0], register[1]);
        return Measure([PauliX], [register[1]]);
    }
}


In [11]:
teleport_message = my_qbob.compile()

In [12]:
teleport_message(message=True)

1

In [13]:
teleport_message(message=False)

0

## Prepare entangled state

In [7]:
from typing import List
from qbob.types import Qubit
from qbob.intrinsics import H, CNOT, M
prepare_qbob = qbob.OperationBuilder("PrepareEntangledState", adj=True)
qubits = prepare_qbob.input("qubits", List[Qubit])

prepare_qbob += H(qubits[0])
prepare_qbob += CNOT(qubits[0], qubits[1])

print(prepare_qbob.build())

operation PrepareEntangledState (qubits : Qubit[]) : Unit is Adj {
    H(qubits[0]);
    CNOT(qubits[0], qubits[1]);
}


In [8]:
measure_qbob = qbob.OperationBuilder("MeasureEntangledState")

with measure_qbob.allocate_qubits("qubits", 2) as q:
    measure_qbob += prepare_qbob(q)
    measure_qbob.returns([M(q[0]), M(q[1])])
    
print(measure_qbob.build())

operation MeasureEntangledState () : Result[] {
    using (qubits = Qubit[2]) {
        PrepareEntangledState(qubits);
        return [M(qubits[0]), M(qubits[1])];
    }
}


In [11]:
prepare_op = prepare_qbob.compile()
measure_op = measure_qbob.compile()

In [12]:
measure_op.simulate()

[0, 0]